In [ ]:
# SIMPLE EARTH SUN SYSTEM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Constants for the elliptical orbit
a = 1.496e11  # Semi-major axis, meters (1 AU)
eccentricity = 0.1  # Eccentricity of the ellipse
b = a * (1 - eccentricity)  # Semi-minor axis

# Orbital period using a simplified version for elliptical orbits (Kepler's third law)
T = 2 * np.pi * np.sqrt(a**3 / (G * M_star))

# Time array for one orbit period
time = np.linspace(0, T, 360)

# Position of the planet in the orbit
x = a * np.cos(2 * np.pi * time / T)
y = b * np.sin(2 * np.pi * time / T)

# Setting up the plot
fig, (ax_orbit, ax_x) = plt.subplots(1, 2, figsize=(10, 5))
ax_orbit.set_xlim(-1.5 * a, 1.5 * a)
ax_orbit.set_ylim(-1.5 * a, 1.5 * a)
ax_orbit.set_aspect('equal', 'box')
ax_orbit.set_facecolor('black')
ax_orbit.set_title('Elliptical Orbit of a Planet Around a Star', color='white')
star = plt.Circle((0, 0), 0.05 * a, color='red', label='Star')
planet = plt.Circle((a, 0), 0.03 * a, color='blue', label='Planet')
orbit_path, = ax_orbit.plot([], [], 'y-', linewidth=1)
ax_orbit.add_patch(star)
ax_orbit.add_patch(planet)

# Setting up the x-coordinate plot
ax_x.set_xlim(0, T)
ax_x.set_ylim(-1.5 * a, 1.5 * a)
ax_x.set_title('X-coordinate of Orbit (Cosine Wave)', color='white')
ax_x.set_facecolor('black')
line, = ax_x.plot([], [], 'y-', label='X-coordinate (cos)', color='yellow')

# Function to update the animation
def update(num):
    planet.center = (x[num], y[num])
    orbit_path.set_data(x[:num+1], y[:num+1])
    line.set_data(time[:num+1], x[:num+1])
    return planet, orbit_path, line

# Creating the animation
ani = FuncAnimation(fig, update, frames=360, blit=True, repeat=True, interval=20)
ani.save('simple.gif', writer='pillow')


# Show the plot with the animation
ax_orbit.legend(handles=[star, planet])
ax_x.legend()
plt.show()


In [ ]:
# SMALL PERTURBATION - EFFECTIVE PICTURE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Constants
G = 6.67430e-11  # Gravitational constant
M_star = 5e30   # Mass of each star
a = 1.496e11    # Semi-major axis of planet's orbit
eccentricity = 0.1  # Eccentricity of the planet's orbit
b = a * np.sqrt(1 - eccentricity**2)
star_orbit_radius = 0.1 * a  # Radius of stars' orbit

# Time settings
T = 365.25 * 24 * 3600  # One orbital period in seconds
dt = 2 * 24 * 3600  # Time step (10 days)
num_orbits = 4  # Number of orbits to simulate
time = np.arange(0, num_orbits * T, dt)

# Positions of the binary stars
theta = 2 * np.pi * time / (0.5 * T)  # Half-year period for binary stars
x_star1 = star_orbit_radius * np.cos(theta)
y_star1 = star_orbit_radius * np.sin(theta)
x_star2 = -x_star1
y_star2 = -y_star1

# Initialize planet position
x_planet = np.zeros_like(time)
y_planet = np.zeros_like(time)
x_planet[0] = a
y_planet[0] = 0

# Velocity of the planet
vx = 0
vy = np.sqrt(G * 2 * M_star / a)

# Numerical integration for planet's orbit
for i in range(1, len(time)):
    r1 = np.sqrt((x_planet[i-1] - x_star1[i-1])**2 + (y_planet[i-1] - y_star1[i-1])**2)
    r2 = np.sqrt((x_planet[i-1] - x_star2[i-1])**2 + (y_planet[i-1] - y_star2[i-1])**2)

    # Gravitational forces
    fx1 = -G * M_star * (x_planet[i-1] - x_star1[i-1]) / r1**3
    fy1 = -G * M_star * (y_planet[i-1] - y_star1[i-1]) / r1**3
    fx2 = -G * M_star * (x_planet[i-1] - x_star2[i-1]) / r2**3
    fy2 = -G * M_star * (y_planet[i-1] - y_star2[i-1]) / r2**3

    # Update velocity and position
    vx += (fx1 + fx2) * dt
    vy += (fy1 + fy2) * dt
    x_planet[i] = x_planet[i-1] + vx * dt
    y_planet[i] = y_planet[i-1] + vy * dt

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 12))
ax1.set_facecolor('black')
ax1.set_xlim(-1.5 * a, 1.5 * a)
ax1.set_ylim(-1.5 * a, 1.5 * a)
ax1.set_aspect('equal')

# Orbit plots
planet_path, = ax1.plot([], [], 'y-', lw=2)  # Line for planet's orbit
star1_path, = ax1.plot(x_star1, y_star1, 'r-', ms=10, label='Star 1')  # Red star path
star2_path, = ax1.plot(x_star2, y_star2, 'r-', ms=10, label='Star 2')  # Red star path
planet_dot, = ax1.plot([], [], 'wo', ms=5)  # Planet

# X-coordinate plots
x_trace_planet, = ax2.plot([], [], 'y-', label='Planet X Coordinate')
x_trace_star1, = ax2.plot([], [], 'r-', label='Star 1 X Coordinate')

def init():
    ax1.set_xlim(-1.5 * a, 1.5 * a)
    ax1.set_ylim(-1.5 * b, 1.5 * b)
    ax2.set_xlim(0, num_orbits * T)
    ax2.set_ylim(-1.5 * a, 1.5 * a)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('X Coordinate')
    ax2.legend()
    return planet_path, star1_path, star2_path, planet_dot, x_trace_planet, x_trace_star1

def update(frame):
    # Update orbital paths and dots
    planet_path.set_data(x_planet[:frame], y_planet[:frame])
    planet_dot.set_data(x_planet[frame], y_planet[frame])
    star1_path.set_data(x_star1[:frame], y_star1[:frame])
    star2_path.set_data(x_star2[:frame], y_star2[:frame])
    
    # Update x-coordinate traces
    x_trace_planet.set_data(time[:frame], x_planet[:frame])
    x_trace_star1.set_data(time[:frame], x_star1[:frame])
    
    return planet_path, star1_path, star2_path, planet_dot, x_trace_planet, x_trace_star1

ani = FuncAnimation(fig, update, frames=len(time), init_func=init, blit=True, repeat=False)
ani.save('3bodyfullfinal.gif', writer='pillow')  # Save the animation as a GIF

plt.show()


In [ ]:
# CHAOTIC ORBITS OF THE 3-BODY PROBLEM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Constants
G = 6.67430e-11  # Gravitational constant
M_star = 5e30   # Mass of each star
a = 1.496e11    # Semi-major axis of planet's orbit
eccentricity = 0.1  # Eccentricity of the planet's orbit
b = a * np.sqrt(1 - eccentricity**2)
star_orbit_radius = 0.2 * a  # Radius of stars' orbit, increased to 0.5 of planet's semi-major axis

# Time settings
T = 365.25 * 24 * 3600  # One orbital period in seconds
dt = 1 * 24 * 3600  # Time step (10 days)
num_orbits = 3  # Number of orbits to simulate
time = np.arange(0, num_orbits * T, dt)

# Positions of the binary stars
theta = 2 * np.pi * time / (0.5 * T)  # Half-year period for binary stars
x_star1 = star_orbit_radius * np.cos(theta)
y_star1 = star_orbit_radius * np.sin(theta)
x_star2 = -x_star1
y_star2 = -y_star1

# Initialize planet position
x_planet = np.zeros_like(time)
y_planet = np.zeros_like(time)
x_planet[0] = a
y_planet[0] = 0

# Velocity of the planet
vx = 0
vy = np.sqrt(G * 2 * M_star / a)

# Numerical integration for planet's orbit
for i in range(1, len(time)):
    r1 = np.sqrt((x_planet[i-1] - x_star1[i-1])**2 + (y_planet[i-1] - y_star1[i-1])**2)
    r2 = np.sqrt((x_planet[i-1] - x_star2[i-1])**2 + (y_planet[i-1] - y_star2[i-1])**2)

    # Gravitational forces
    fx1 = -G * M_star * (x_planet[i-1] - x_star1[i-1]) / r1**3
    fy1 = -G * M_star * (y_planet[i-1] - y_star1[i-1]) / r1**3
    fx2 = -G * M_star * (x_planet[i-1] - x_star2[i-1]) / r2**3
    fy2 = -G * M_star * (y_planet[i-1] - y_star2[i-1]) / r2**3

    # Update velocity and position
    vx += (fx1 + fx2) * dt
    vy += (fy1 + fy2) * dt
    x_planet[i] = x_planet[i-1] + vx * dt
    y_planet[i] = y_planet[i-1] + vy * dt

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 12))
ax1.set_facecolor('black')
ax1.set_xlim(-1.5 * a, 1.5 * a)
ax1.set_ylim(-1.5 * a, 1.5 * a)
ax1.set_aspect('equal')

# Orbit plots
planet_path, = ax1.plot([], [], 'y-', lw=2)  # Line for planet's orbit
star1_path, = ax1.plot(x_star1, y_star1, 'r-', ms=10, label='Star 1')  # Red star path
star2_path, = ax1.plot(x_star2, y_star2, 'r-', ms=10, label='Star 2')  # Red star path
planet_dot, = ax1.plot([], [], 'wo', ms=5)  # Planet

# X-coordinate plots
x_trace_planet, = ax2.plot([], [], 'y-', label='Planet X Coordinate')
x_trace_star1, = ax2.plot([], [], 'r-', label='Star 1 X Coordinate')

def init():
    ax1.set_xlim(-1.5 * a, 1.5 * a)
    ax1.set_ylim(-1.5 * b, 1.5 * b)
    ax2.set_xlim(0, num_orbits * T)
    ax2.set_ylim(-1.5 * a, 1.5 * a)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('X Coordinate')
    ax2.legend()
    return planet_path, star1_path, star2_path, planet_dot, x_trace_planet, x_trace_star1

def update(frame):
    planet_path.set_data(x_planet[:frame], y_planet[:frame])
    planet_dot.set_data(x_planet[frame], y_planet[frame])
    star1_path.set_data(x_star1[:frame], y_star1[:frame])
    star2_path.set_data(x_star2[:frame], y_star2[:frame])
    x_trace_planet.set_data(time[:frame], x_planet[:frame])
    x_trace_star1.set_data(time[:frame], x_star1[:frame])
    return planet_path, star1_path, star2_path, planet_dot, x_trace_planet, x_trace_star1

ani = FuncAnimation(fig, update, frames=len(time), init_func=init, blit=True, repeat=False)
ani.save('3bodychaos1.gif', writer='pillow')  # Save the animation as a GIF

plt.show()
